In [5]:
import pandas as pd
import os
import glob
import difflib

In [6]:

df1=pd.read_excel(f'/workspaces/vikrant_dubai/new/NCM -  May 2017.xlsx')

new_col=[]
columns = df1.columns
for i in range(1, len(columns)):
    if 'Unnamed' in columns[i]:
        continue
    else:
        new_col.append(columns[i]) 

xls = pd.ExcelFile(f"/workspaces/vikrant_dubai/new/NCM -  May 2017.xlsx")

# Load the specific sheet into a DataFrame
df = pd.read_excel(xls, sheet_name='1hr')


for a in range(len(new_col)):
    if a < len(new_col) and new_col[a] in df.columns:
        if a + 1 < len(new_col) and new_col[a + 1] in df.columns:
            index_col_a = df.columns.get_loc(new_col[a])
            index_col_b = df.columns.get_loc(new_col[a + 1])
        elif a - 1 >= 0 and new_col[a - 1] in df.columns:
            index_col_b = df.columns.get_loc(new_col[a])
            index_col_a = df.columns.get_loc(new_col[a - 1])
        
        total_columns_between = index_col_b - index_col_a
        col_index = df.columns.get_loc(new_col[a])
        related_cols = df.iloc[:, col_index: col_index + total_columns_between]

    related_cols_df = pd.DataFrame(related_cols)

    vv = df['1hr'].drop([0, 1]).reset_index(drop=True).to_frame(name='Date')


    station_name=related_cols.columns[0].replace(' ', '')
    related_cols.columns = related_cols.iloc[0]
    related_cols = related_cols[1:]
    related_cols.drop([1], inplace=True)
    related_cols.reset_index(drop=True, inplace=True)
    related_cols['Station Name'] = station_name
    related_cols['Date/Time'] = vv['Date']

    mdf = related_cols

    columns_of_main_df = ['Station Name','Date', 'Time', 'SO2', 'Data Cap. (SO2)', 'SO2 Avg', 'NO2','NOX','NO', 'Data Cap. (NO2)', 'NO2 Avg',      'O3', 'Data Cap. (O3)', 'O3 8H Avg.', 'Daily Capture O3', 'O3 Daily Average', 'O3 Daily Average (NS)', 'CO',  'Data Cap. (CO)', 'CO 8H Avg.', 'Daily Capture CO', 'CO Daily Average', 'PM10',    'Data Cap. (PM10)', 'PM10 Avg', 'PM2.5', 'Data Cap. (PM2.5)', 'PM2.5 Avg', 'Lower Ambient Temperature', 'Upper Ambient Temperature', 'Barometric Pressure', 'Relative Humidity', 'Wind Direction', 'Wind Speed', 'H2S', 'Toluene', 'O-Xylene', 'Ethylbenzene', 'MP-Xylene', 'Benzene', 'CH4', 'NMHC', 'THC', 'Noise', 'Solar Radiation','Atm P.']
    main_df = pd.DataFrame(columns=columns_of_main_df)
    mdf['Date/Time'] = pd.to_datetime(mdf['Date/Time'], format='%d/%m/%Y %H:%M', dayfirst=True)
    mdf['Date'] = mdf['Date/Time'].dt.date
    mdf['Time'] = mdf['Date/Time'].dt.time
    new_df_columns = mdf.columns.tolist()

    mdf = mdf.drop(columns=['Date/Time'])
    mdf['Date'] = pd.to_datetime(mdf['Date']).dt.strftime('%d/%m/%Y')

    values = [col.upper() for col in mdf.columns]
    l1 = [col.upper() for col in main_df.columns]

    matches = {}
    for value in values:
        closest_match = difflib.get_close_matches(value, l1, n=1)
        if closest_match:
            main_df[main_df.columns[main_df.columns.str.upper() == closest_match[0]][0]] = mdf[mdf.columns[mdf.columns.str.upper() == value][0]]
        else:
            main_df[value] = ''

    main_df.to_csv(f'/workspaces/vikrant_dubai/Transformed/{station_name}.csv', index=False)

# Get a list of all CSV files in the transformed folder
csv_files = [f for f in os.listdir('/workspaces/vikrant_dubai/Transformed') if f.endswith('.csv')]

# Read and concatenate all CSV files
combined_df = pd.concat([pd.read_csv(os.path.join('/workspaces/vikrant_dubai/Transformed', f)) for f in csv_files], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(f'/workspaces/vikrant_dubai/final/NCM/NCM 2015.csv', index=False)


# Get a list of all files in the Transformed folder
files = glob.glob('/workspaces/vikrant_dubai/Transformed/*')

# Iterate over the list of files and remove each one
for f in files:
    os.remove(f)
